# CartPole-v0
"A pole is attached by an un-actuated joint to a cart, which moves along a frictionless track. The system is controlled by applying a force of +1 or -1 to the cart. The pendulum starts upright, and the goal is to prevent it from falling over. A reward of +1 is provided for every timestep that the pole remains upright. The episode ends when the pole is more than 15 degrees from vertical, or the cart moves more than 2.4 units from the center."


https://github.com/openai/gym/wiki/MountainCar-v0#solved-requirements :

## Observation
Type: Box(2)
| Num | Observation | Min   | Max  |
|-----|-------------|-------|------|
| 0   | position    | -1.2  | 0.6  |
| 1   | velocity    | -0.07 | 0.07 |

## Action
Type: Discrete(3)
| Num | Action     |
|-----|------------|
| 0   | push left  |
| 1   | no push    |
| 2   | push right |

## Reward
-1 for each time step, until the goal position of 0.5 is reached. As with MountainCarContinuous v0, there is no penalty for climbing the left hill, which upon reached acts as a wall.

## Starting State
Random position from -0.6 to -0.4 with no velocity.

## Episode Termination
The episode ends when you reach 0.5 position, or if 200 iterations are reached.

## Solved Requirements
MountainCar-v0 defines "solving" as getting average reward of -110.0 over 100 consecutive trials.

https://gym.openai.com/envs/CartPole-v0/ &  https://gym.openai.com/docs/ :

Loop: Each timestep, the agent chooses an action, the environment returns an observation and a reward. The process starts by calling reset() which returns an intitial observation:
    "on each episode, and at every step, an action is chosen based on the current state and a policy based on the action-value function Q. After taking the action, we receive a reward and arrive at the next state. This information is used to update the action-value function, and, after doing so, we make the next state our current state and follow through until we reach the final state of the final episode." - https://medium.com/@flomay/using-q-learning-to-solve-the-cartpole-balancing-problem-c0a7f47d3f9d

**Step** returns 4 values:
1. Observation is an environemnt specific object representing your observation of the environment e.g. pixel data from camera or board state in boardgame. 
2. Reward is a float. It is the amout of reward achieved by the prior action. The goal is to increase total reward. The reward scale varies between environments.
3. Done is a boolean. Whether it is time to reset the environment. Most tasks are divided into episodes which terminate when done.
4. Info (dict) provides diagnostic information for debugging.

Every **environment** comes with an action_space and an observation_space, which describe the format of valid actions and observations. The Discrete space allows a fixed range of non-negative numbers e.g. valid **actions** are either 0 or 1.  The Box space represents and n-dimensional box so valid **observations** will be an array of 4 numbers.

In CartPole, one of the actions applies force to the left and one applies force to the right

"Since this [Q learning] algorithm relies on updating a function for each existing pair of state and action, environments that have a high state-space become problematic. This is because we can approximate better the actual value of a state-action pair as we visit it more often. However, if we have many states or many actions to take, we distribute our visits among more pairs and it takes much longer to converge to the actual true values. " - https://medium.com/@flomay/using-q-learning-to-solve-the-cartpole-balancing-problem-c0a7f47d3f9d 

The action-value funciton must be updated at every step of learning. The values for hte pairs of state visited and action taken are updated. 
 
See also: https://medium.com/swlh/using-q-learning-for-openais-cartpole-v1-4a216ef237df
Read https://medium.com/@flomay/using-q-learning-to-solve-the-cartpole-balancing-problem-c0a7f47d3f9d with https://github.com/JoeSnow7/Reinforcement-Learning/blob/master/Cartpole%20Q-learning.ipynb

#### SUMMARY
Environment: The whole grid world including all squares e.g. 4x4 = 16 squares
State: One square
Agent: Can occupy one state at a time, can perform actions (up, down, left, right). As the agent travels and explores the environment it will work out th ebest policy.
Reward: +1 when finding the fisbee goal, -1 when falling in a hole.Can implement a negative reward eg -0.1 per step i.e. higher reward if reached faster.
Policy: Map & information about what action to take in a particular state. The best policy is to thake the best action in a particular state. Find a policy that allows agent to gain maxium reward.

###### Q-TABLE (reward table)
As the agent expolores, it records the best action to take for each state (square) in the Q-Table 
e.g. If you start in state C1 and the reward is at A1, it is best to move up when in square C1. From B1 it is then best to move up and get the reward at A1.

###### Goal: The way to choose the best action fr every state in the environment.
- Gym presets the reward ie cannot change reward to solve these problems. 
- You can change how the q-table is populated. You can change how the Q-table is calculated on each step. e.g. Keep track of the reward for each step and long term.



In [1]:
import numpy as np #for array manipulation
import gym # pull the cart pole environment from Open AI.
#import random
import time
import math
#from IPython.display import clear_output


import matplotlib.pyplot as plt
import gc
gc.disable() #Disable automatic garbage collection.

In [2]:
#Create environment
import gym
env = gym.make('CartPole-v0') 
# Every environment comes with an action_space and an observation_space. These attributes are of type Space, and they describe the format of valid actions and observations:
print(env.action_space) #> Discrete(2)
print(env.observation_space) #> Box(4,)
#The Discrete space allows a fixed range of non-negative numbers, so in this case valid actions are either 0 or 1. The Box space represents an n-dimensional box, so valid observations will be an array of 4 numbers. We can also check the Box’s bounds:
print(env.observation_space.high)  #> array([ 2.4       ,         inf,  0.20943951,         inf])
print(env.observation_space.low)  #> array([-2.4       ,        -inf, -0.20943951,        -inf])

#You can sample from a Space or check that something belongs to it:
# from gym import spaces
# space = spaces.Discrete(8) # Set with 8 elements {0, 1, 2, ..., 7}
# x = space.sample()
# assert space.contains(x)
# assert space.n == 8


Discrete(2)
Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)
[4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38]
[-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38]


In [3]:
#This will run an instance of the CartPole-v0 environment for however many timesteps, rendering the environment at each step.
#The process gets started by calling reset(), which returns an initial observation. 
for episode in range(20): #number of episodes
    observation = env.reset()
    for t in range(100): #amount of time
        env.render()
        print(observation)
        action = env.action_space.sample() # take a random action
        observation, reward, done, info = env.step(action)
        if done:
            print("Episode finished after {} timesteps".format(t+1))
            break
env.close()

[ 0.04394225 -0.0188086   0.03583074 -0.00304046]
[ 0.04356608  0.17578165  0.03576994 -0.28420651]
[ 0.04708171  0.37037565  0.03008581 -0.56539668]
[ 0.05448922  0.56506288  0.01877787 -0.84845148]
[ 0.06579048  0.36968992  0.00180884 -0.54992333]
[ 0.07318428  0.17454261 -0.00918962 -0.25667104]
[ 0.07667513 -0.02044694 -0.01432305  0.03309923]
[ 0.07626619  0.17487744 -0.01366106 -0.26406814]
[ 0.07976374 -0.02004688 -0.01894242  0.02427484]
[ 0.0793628   0.17534153 -0.01845693 -0.27432393]
[ 0.08286964  0.37072189 -0.02394341 -0.57277059]
[ 0.09028407  0.1759437  -0.03539882 -0.28772574]
[ 0.09380295 -0.01865603 -0.04115333 -0.00641409]
[ 0.09342983  0.17703122 -0.04128161 -0.3117921 ]
[ 0.09697045  0.37271624 -0.04751746 -0.61720289]
[ 0.10442478  0.56846866 -0.05986151 -0.92446498]
[ 0.11579415  0.37420416 -0.07835081 -0.65117894]
[ 0.12327823  0.18025584 -0.09137439 -0.38416054]
[ 0.12688335 -0.01345807 -0.0990576  -0.12162856]
[ 0.12661419  0.18293326 -0.10149017 -0.44384677]
